In [1]:
import requests

In [2]:
url = 'http://www.happymoney.co.kr/svc/store/offlineStore.hm#quick'

In [3]:
from bs4 import BeautifulSoup

In [4]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
options.add_argument("lang=ko_KR")

In [5]:
driver = webdriver.Chrome('../.dev/bin/chromedriver')

In [ ]:
driver.get(url)

In [ ]:
while True:
    try:
        btn = driver.find_element_by_css_selector('button.btnMore.js-more')
        btn.click()
    except NoSuchElementException:
        break

In [ ]:
elem = driver.find_element_by_css_selector('ul.imgListWrap')

In [ ]:
html = elem.get_attribute('innerHTML')

In [ ]:
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
item_lists = soup.findChildren('li')

In [ ]:
import os

In [ ]:
curdir = os.path.abspath(os.path.curdir)
imgdir = os.path.join(curdir, 'site-images')
jsonfile = os.path.join(curdir, 'sites.json')

In [ ]:
if not os.path.isdir(imgdir):
    os.mkdir(imgdir)

In [ ]:
import re

In [ ]:
# 온라인점 크롤링

ignorecase = re.compile(r'\w+/useStore/(.*)', re.IGNORECASE)
regex_import_store_number = re.compile(r"'(\w+)'")
regex_import_site_url = re.compile(r"'(http.*)',")

import_store_number = 0
import_url_front = f"http://www.happymoney.co.kr/svc/store/useStoreView.hm?useStoreInfoId="
import_url_back = "&pageLink=store/offlineStore.hm"

for item in item_lists:
    img_url = item.select_one('img')['src']
    
    img_name = ignorecase.search(img_url).group(1)
    
    r = requests.get(img_url)
    img_path = os.path.join(imgdir, img_name)
    print(img_path)

    if r.status_code == 200:
        with open(img_path, 'wb') as f:
            f.write(r.content)
    
    site_name = item.select_one('div.info > a').get_text()
    category = (item.select_one('div.info > span').get_text() + '/').split('/')[0]
    is_fee = True if item.select_one('strong.ir.ico.feesIcon') else False
    is_import_point = True if item.select_one('strong.ir.ico.storeLab') else False
    month_pay_limit = 0
    # 입점몰의 site_url은 다른 사이트의 iframe을 요구하기때문에 구현하지 않는다.
    site_url = ''
    
    if is_import_point:
        import_store_number = regex_import_store_number.search(item.select_one('div.info > a')['onclick']).group(1)
        url = import_url_front + import_store_number + import_url_back
        driver.get(url)
        month_pay_limit = driver.find_element_by_css_selector('span#paymentMonthLimit').text
    else:
        site_url = item.select_one('span.btn > a')['href']
    